# Search to Network Graph With Pandas, NetworkX and Sigma (ipysigma)

This Jupyternotebook demonstrates how to create & visualize a graph based on urlscan.io search results.

## How It Works

- Do search
- Convert search results into a Pandas DataFrame
- Convert the Pandas DataFrame into a NetworkX graph represents `page.ip` to `page.domain` relationships
- Visualize the graph with Sigma.js (`ipysigma`)

In [ ]:
!pip install python-dotenv networkx pandas ipysigma urlscan-python

In [ ]:
import os

import dotenv
import networkx as nx
import pandas as pd
from ipysigma import Sigma

from urlscan import Client

In [ ]:
# load urlscan.API key via URLSCAN_API_KEY env or input & set search query via input
dotenv.load_dotenv()

URLSCAN_API_KEY = os.environ.get("URLSCAN_API_KEY") or input("URLSCAN_API_KEY: ")
SEARCH_QUERY = input("Search query:")

In [ ]:
# do search
print(f"Searching for {SEARCH_QUERY}...")  # noqa: T201

with Client(URLSCAN_API_KEY) as client:
    results = list(client.search(SEARCH_QUERY, limit=10000))

In [ ]:
# convert search results to pandas DataFrame
df = pd.json_normalize(results)

In [ ]:
# drop duplicates
df = df.drop_duplicates(subset=["page.ip", "page.domain"])

In [ ]:
# convert DataFrame to networkx graph
g = nx.Graph()
g.add_nodes_from(df["page.ip"], tag="ip")
g.add_nodes_from(df["page.domain"], tag="domain")
edges = [(row["page.ip"], row["page.domain"]) for _, row in df.iterrows()]
g.add_edges_from(edges)

In [ ]:
# visualize graph with ipysigma
Sigma(g, node_color="tag")